In [1]:
import nltk
from nltk.stem.porter import *
from nltk.classify import MaxentClassifier
import pickle
import os,sys
from io import open
import numpy as np

In [2]:
change_of_sentence_flag = 0 #a marker for the end of sentence
boi_full_list = [] #store all the boi tags that occur in the training set
boi_end_list = [] #store boi tags that are at the end of the sentence
wordStartList = [] #store words that are begining of the sentence
BOI_list = ['B', 'I', 'O']
labeled_features = []


In [5]:
training_file = open("../assignment2dataset/train.txt", "r")
previous_BOI = "start"
input_file = training_file
for line in input_file:
    s = re.match(r'^\s*$', line)  #find empty line
    if s:
        change_of_sentence_flag = 1
        previous_BOI = "start"

    else: 
        sentenceList = line.split()
        word = sentenceList[0]
        tag = sentenceList[1]
        boi = sentenceList[2][0]

        #store words that are begining of the sentence
        if change_of_sentence_flag == 1:
            wordStartList.append(word)
            boi_end_list.append(boi_full_list[-1])
            change_of_sentence_flag = 0
        boi_full_list.append(boi)
        item = word, tag, boi, previous_BOI
        labeled_features.append(item)
        previous_BOI = boi
input_file.close()
labeled_features=[('start','X','O'),('start','X','O')]+labeled_features+[('start','X','O'),('start','X','O')]
print(boi_full_list)
print(labeled_features)
print(len(boi_full_list))
print(boi_end_list)
print(wordStartList)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [85]:
def isSubstring(s1,s2):
    s1=s1.lower()
    s1=s1.strip()
    s2=s2.strip()
    for i in range(len(s1)-len(s2)+1):
        if s1[i:i+len(s2)]==s2:
            return 1
    return 0
def featuring(w,t):
    word=w
    tag=t
    l=[]
    n=isSubstring(word,'eer')
    l.append(n)
    n=isSubstring(word,'er')
    l.append(n)
    n=isSubstring(word,'ion')
    l.append(n)
    n=isSubstring(word,'ity')
    l.append(n)
    n=isSubstring(word,'ment')
    l.append(n)
    n=isSubstring(word,'ness')
    l.append(n)
    n=isSubstring(word,'or')
    l.append(n)
    n=isSubstring(word,'sion')
    l.append(n)
    n=isSubstring(word,'th')
    l.append(n)
    n=isSubstring(word,'ship')
    l.append(n)
    
    ###tag based###
    if tag=='ADJ':
        l.append(1)
    else:
        l.append(0)
    if tag=='ADP':
        l.append(1)
    else:
        l.append(0)
    if tag=='ADV':
        l.append(1)
    else:
        l.append(0)
    if tag=='AUX':
        l.append(1)
    else:
        l.append(0)
    if tag=='CCONJ':
        l.append(1)
    else:
        l.append(0)
    if tag=='DET':
        l.append(1)
    else:
        l.append(0)
    if tag=='INTJ':
        l.append(1)
    else:
        l.append(0)
    if tag=='NOUN':
        l.append(1)
    else:
        l.append(0)
    if tag=='NUM':
        l.append(1)
    else:
        l.append(0)
    if tag=='PART':
        l.append(1)
    else:
        l.append(0)
    if tag=='PRON':
        l.append(1)
    else:
        l.append(0)
    if tag=='PROPN':
        l.append(1)
    else:
        l.append(0)
    if tag=='PUNCT':
        l.append(1)
    else:
        l.append(0)
    if tag=='SCONJ':
        l.append(1)
    else:
        l.append(0)
    if tag=='SYM':
        l.append(1)
    else:
        l.append(0)
    if tag=='VERB':
        l.append(1)
    else:
        l.append(0)
    if tag=='X':
        l.append(1)
    else:
        l.append(0)
    
    return l
def MEMM_features(word, tag, previous_BOI,prev_prev_tup,prev_tup,next_tup,next_next_tup):
   # print(word)
    stemmer = PorterStemmer() 
    features = {} 
    mainlist=[]
    features['current_word'] = word
    features['current_tag'] = tag
    ft=featuring(prev_prev_tup[0],prev_prev_tup[1])+featuring(prev_tup[0],prev_tup[1])+featuring(word,tag)+featuring(next_tup[0],next_tup[1])+featuring(next_next_tup[0],next_next_tup[1])
    a=np.array(ft)
    a=np.matrix(a)
    features['history']=a
     #some char is outof ASCII
   # print (word)
    features['capitalization'] = word[0].isupper()
    features['start_of_sentence'] = word in wordStartList
    features['cap_start'] = word not in wordStartList and word[0].isupper()
    features['previous_NC'] = previous_BOI
    return features

In [87]:
#labeled_featuresets = [(MEMM_features(word, tag, previous_BOI), boi )for (word, tag, boi, previous_BOI) in labeled_features]
tup=()
#print(labeled_features)
labelled_featuresets=[]
temp=()
for i in range(2,len(labeled_features)-2,1):
    tup=MEMM_features(labeled_features[i][0],labeled_features[i][1],labeled_features[i][3],labeled_features[i-2],labeled_features[i-1],labeled_features[i+1],labeled_features[i+2])
    temp=(tup,labeled_features[i][2])
    labelled_featuresets.append(temp)
train_set = labelled_featuresets
print(labelled_featuresets)

[({'current_word': 'Confidence', 'current_tag': 'NN', 'history': matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'capitalization': True, 'start_of_sentence': False, 'cap_start': True, 'previous_NC': 'start'}, 'B'), ({'current_word': 'in', 'current_tag': 'IN', 'history': matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 

In [88]:
maxent_classifier = MaxentClassifier.train(train_set, max_iter=30)

TypeError: unhashable type: 'matrix'